# Module 1: Introduction
## Homework

In this homework, we'll learn more about search and use Elastic Search for practice. Original description of tasks is [here](https://github.com/DataTalksClub/llm-zoomcamp/blob/main/cohorts/2024/01-intro/homework.md).

In [104]:
import os
from dotenv import load_dotenv
import requests
import json
from elasticsearch import Elasticsearch
from tqdm.auto import tqdm

from openai import OpenAI
import tiktoken

from mistralai.client import MistralClient
from mistralai.models.chat_completion import ChatMessage

from groq import Groq



In [105]:
load_dotenv()

True

## Q1. Running Elastic 

Run Elastic Search 8.4.3, and get the cluster information. If you run it on localhost, this is how you do it:

```bash
curl localhost:9200
```

What's the `version.build_hash` value?


Run in terminal in prepared environment

```bash
docker run -it \
    --rm \
    --name elasticsearch \
    -p 9200:9200 \
    -p 9300:9300 \
    -e "discovery.type=single-node" \
    -e "xpack.security.enabled=false" \
    docker.elastic.co/elasticsearch/elasticsearch:8.4.3
```

In [1]:
!curl localhost:9200

{
  "name" : "d5654c2a8176",
  "cluster_name" : "docker-cluster",
  "cluster_uuid" : "aEdQDmqQSiK3n2f6ro4dKA",
  "version" : {
    "number" : "8.4.3",
    "build_flavor" : "default",
    "build_type" : "docker",
    "build_hash" : "42f05b9372a9a4a470db3b52817899b99a76ee73",
    "build_date" : "2022-10-04T07:17:24.662462378Z",
    "build_snapshot" : false,
    "lucene_version" : "9.3.0",
    "minimum_wire_compatibility_version" : "7.17.0",
    "minimum_index_compatibility_version" : "7.0.0"
  },
  "tagline" : "You Know, for Search"
}


In [4]:

response = requests.get('http://localhost:9200/')

if response.status_code == 200:
    data = response.json()
    build_hash = data['version']['build_hash']
    print(f"version.build_hash: {build_hash}")
else:
    print(f"Failed to connect to Elasticsearch: {response.status_code}")


version.build_hash: 42f05b9372a9a4a470db3b52817899b99a76ee73


## Q2. Indexing the data

Index the data in the same way as was shown in the course videos. Make the `course` field a keyword and the rest should be text. 

Don't forget to install the ElasticSearch client for Python:

```bash
pip install elasticsearch
```

Which function do you use for adding your data to elastic?

* `insert`
* `index`
* `put`
* `add`

### Loading data from prepared json file

In [21]:
dataset_url = "https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/01-intro/documents.json"
dataset_file = dataset_url.split("/")

In [26]:
data_directory = "data"
dataset_url = "https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/01-intro/documents.json"
dataset_file = dataset_url.split("/")[-1]
dataset_path = f"{data_directory}/{dataset_file}"
os.makedirs(data_directory, exist_ok=True)

In [27]:
# Original dataset – https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json

!wget -O {dataset_path} {dataset_url}


--2024-06-22 21:17:35--  https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/01-intro/documents.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 2606:50c0:8000::154, 2606:50c0:8001::154, 2606:50c0:8002::154, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|2606:50c0:8000::154|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 658332 (643K) [text/plain]
Saving to: ‘data/documents.json’

data/documents.json 100%[===================>] 642.90K  3.52MB/s    in 0.2s    

2024-06-22 21:17:36 (3.52 MB/s) - ‘data/documents.json’ saved [658332/658332]



In [31]:
with open(dataset_path, 'r') as file:
    documents_raw = json.load(file)

documents = []

for course in documents_raw:
    course_name = course["course"]
    
    for doc in course["documents"]:
        doc["course"] = course_name
        documents.append(doc)

In [32]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

### Indexing the data by ElasticSearch

In [34]:
from elasticsearch import Elasticsearch

es_client = Elasticsearch('http://localhost:9200') 

es_client.info()

ObjectApiResponse({'name': 'd5654c2a8176', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'aEdQDmqQSiK3n2f6ro4dKA', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [35]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [36]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

100%|██████████| 948/948 [00:02<00:00, 372.58it/s]


## Q3. Searching

Now let's search in our index. 

We will execute a query "How do I execute a command in a running docker container?". 

Use only `question` and `text` fields and give `question` a boost of 4, and use `"type": "best_fields"`.

What's the score for the top ranking result?

* 94.05
* 84.05
* 74.05
* 64.05

Look at the `_score` field.

In [39]:
query = "How do I execute a command in a running docker container?"

search_query = {
    "size": 5,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text"],    # ^4 mean that question is 3 times more important + we dont use "section" here in list
                    "type": "best_fields"
                }
            },
            # "filter": {
            #     "term": {
            #         "course": "data-engineering-zoomcamp"
            #     }
            # }
        }
    }
}

response = es_client.search(index=index_name, body=search_query)

response['hits']['hits'][0]["_score"]

84.050095

## Q4. Filtering

Now let's only limit the questions to `machine-learning-zoomcamp`.

Return 3 results. What's the 3rd question returned by the search engine?

* How do I debug a docker container?
* How do I copy files from a different folder into docker container’s working directory?
* How do Lambda container images work?
* How can I annotate a graph?

In [42]:
query = "How do I execute a command in a running docker container?"

search_query = {
    "size": 3,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text"],    # ^4 mean that question is 3 times more important + we dont use "section" here in list
                    "type": "best_fields"
                }
            },
            "filter": {
                "term": {
                    "course": "machine-learning-zoomcamp"
                }
            }
        }
    }
}

response = es_client.search(index=index_name, body=search_query)

response['hits']['hits'][2]['_source']['question']

'How do I copy files from a different folder into docker container’s working directory?'

## Q5. Building a prompt

Now we're ready to build a prompt to send to an LLM. 

Take the records returned from Elasticsearch in Q4 and use this template to build the context. Separate context entries by two linebreaks (`\n\n`)
```python
context_template = """
Q: {question}
A: {text}
""".strip()
```

Now use the context you just created along with the "How do I execute a command in a running docker container?" question 
to construct a prompt using the template below:

```
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()
```

What's the length of the resulting prompt? (use the `len` function)

* 962
* 1462
* 1962
* 2462


In [44]:
response

ObjectApiResponse({'took': 24, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 345, 'relation': 'eq'}, 'max_score': 84.050095, 'hits': [{'_index': 'course-questions', '_id': 'EoAvQZABbXebgK4PqK-Z', '_score': 84.050095, '_source': {'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)', 'section': '5. Deploying Machine Learning Models', 'question': 'How do I debug a docker container?', 'course': 'machine-learning-zoomcamp'}}, {'_index': 'course-questions', '_id': 'MYAvQZABbXebgK4PqK_g', '_score': 51.04628, '_source': {'text': "You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:\nTo 

In [69]:
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

context_template = """
Q: {question}
A: {text}
""".strip()

context = ""

for hit in response['hits']['hits']:
    context = context + context_template.format(question=hit['_source']['question'], text=hit['_source']['text']) + "\n\n"

prompt = prompt_template.format(question=query, context=context).strip()
len(prompt)

1462


## Q6. Tokens

When we use the OpenAI Platform, we're charged by the number of 
tokens we send in our prompt and receive in the response.

The OpenAI python package uses `tiktoken` for tokenization:

```bash
pip install tiktoken
```

Let's calculate the number of tokens in our query: 

```python
encoding = tiktoken.encoding_for_model("gpt-4o")
```

Use the `encode` function. How many tokens does our prompt have?

* 122
* 222
* 322
* 422

Note: to decode back a token into a word, you can use the `decode_single_token_bytes` function:

```python
encoding.decode_single_token_bytes(63842)
```

In [71]:
encoding = tiktoken.encoding_for_model("gpt-4o")

len(encoding.encode(prompt))

322

In [72]:
encoding.decode_single_token_bytes(63842)

b"You're"

## Bonus: generating the answer (ungraded)

Let's send the prompt to OpenAI. What's the response?  

Note: you can replace OpenAI with Ollama. See module 2.

### → Use OpenAI

In [110]:
%%time 

client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))

response = client.chat.completions.create(
    model='gpt-4o',
    messages=[{"role": "user", "content": prompt}]
)

answer = response.choices[0].message.content
print(answer, '\n')

To execute a command in a running Docker container, follow these steps:

1. Find the container ID of the running container using `docker ps`:
   ```sh
   docker ps
   ```
   This will list all running containers along with their container IDs.

2. Once you have the container ID, you can execute a command in the specific container using `docker exec`. For example, to open a bash shell in the container:
   ```sh
   docker exec -it <container-id> bash
   ``` 

CPU times: user 39.5 ms, sys: 6.84 ms, total: 46.4 ms
Wall time: 2.02 s


### → Use GROQ
https://console.groq.com/keys

In [109]:
%%time 

client = Groq(
    api_key=os.environ.get("GROQ_API_KEY"),
)

response = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": prompt,
        }
    ],
    model="llama3-8b-8192",
)
answer = response.choices[0].message.content

print(answer, '\n')


Based on the context from the FAQ database, to execute a command in a running Docker container, you can use the following command:

docker exec -it <container-id> bash

Where <container-id> is the ID of the specific container you found using the command "docker ps". 

CPU times: user 34.3 ms, sys: 4.18 ms, total: 38.5 ms
Wall time: 5.28 s


### → Use Mistral
https://console.mistral.ai/api-keys/

In [111]:
%%time 

client = MistralClient(api_key=os.getenv('MISTRAL_API_KEY'))

response = client.chat(
    model='open-mistral-7b',
    messages=[
        ChatMessage(role="user", content=prompt)
    ]
)

answer = response.choices[0].message.content
print(answer, '\n')

To execute a command in a running Docker container, follow these steps:

1. First, find the container ID using the `docker ps` command. This command lists all running Docker containers.

2. After finding the container ID, execute the command in the specific container using the `docker exec` command:

```bash
docker exec -it <container-id> <command>
```

Replace `<container-id>` with the ID of your container, and `<command>` with the command you want to execute. For example, if you want to start a bash shell:

```bash
docker exec -it <container-id> bash
```

This command will open a bash shell in the specified container, allowing you to execute any further commands. 

CPU times: user 36.4 ms, sys: 3.01 ms, total: 39.4 ms
Wall time: 2.06 s


### → Use google/gemma-1.1-7b-it by HuggingFace
https://huggingface.co/google/gemma-1.1-7b-it

In [112]:
%%time 

API_URL = "https://api-inference.huggingface.co/models/google/gemma-1.1-7b-it"
headers = {
    "Authorization": f"Bearer {os.getenv('HUGGING_FACE_API_TOKEN')}",
    "Content-Type": "application/json"
}

response = requests.post(API_URL, headers=headers, json={"inputs":prompt})
answer = response.json()[0]['generated_text'].split('**Answer:**')[-1]

print(answer, '\n')




To execute a command in a running docker container, use the `docker exec -it <container-id> bash` command. 

CPU times: user 5.04 ms, sys: 2.5 ms, total: 7.55 ms
Wall time: 423 ms


### → Use meta-llama/Meta-Llama-3-8B-Instruct by HuggingFace
https://huggingface.co/meta-llama/Meta-Llama-3-8B-Instruct

In [113]:
%%time 

API_URL = "https://api-inference.huggingface.co/models/google/gemma-1.1-7b-it"
headers = {
    "Authorization": f"Bearer {os.getenv('HUGGING_FACE_API_TOKEN')}",
    "Content-Type": "application/json"
}

response = requests.post(API_URL, headers=headers, json={"inputs":prompt})
answer = response.json()[0]['generated_text'].split('**Answer:**')[-1]

print(answer, '\n')


To execute a command in a running docker container, use the `docker exec -it <container-id> bash` command. 

CPU times: user 7.43 ms, sys: 2.36 ms, total: 9.79 ms
Wall time: 380 ms


## Bonus: calculating the costs (ungraded)

Suppose that on average per request we send 150 tokens and receive back 250 tokens.

How much will it cost to run 1000 requests?

You can see the prices [here](https://openai.com/api/pricing/)

On June 17, the prices for gpt4o are:

* Input: $0.005 / 1K tokens
* Output: $0.015 / 1K tokens

You can redo the calculations with the values you got in Q6 and Q7.

In [117]:
avg_request_tokens = 150
avg_response_tokens = 250
n_requests = 1000


request_token_price_gpt4o = 0.005 / 1000
response_token_price_gpt4o = 0.015 / 1000

n_price = n_requests * (
    avg_request_tokens * request_token_price_gpt4o
    + avg_response_tokens * response_token_price_gpt4o
)
print(f"Total price for GPT-4o = {n_price:.2f}")

request_token_price_gpt35turbo = 0.005 / 10000
response_token_price_gpt35turbo = 0.015 / 10000

n_price = n_requests * (
    avg_request_tokens * request_token_price_gpt35turbo
    + avg_response_tokens * response_token_price_gpt35turbo
)
print(f"Total price for GPT-3.5 Turbo = {n_price:.2f}")



Total price for GPT-4o = 4.50
Total price for GPT-3.5 Turbo = 0.45
